<a href="https://colab.research.google.com/github/naolnegassa/Auto_Classify/blob/master/Auto_quer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install google_images_download
#Imports
import tensorflow as tf
import keras
from google.colab import drive
import os
from fastai.vision import *
from fastai.metrics import error_rate
import re
from google_images_download import google_images_download

In [0]:
# Start off with Mounting Drive Locally
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
!ls "/content/drive/My Drive/Auto_Query"

In [0]:
#change the working directory to the Drive folder
import os
os.chdir("/content/drive/My Drive/Auto_Query")
#Initiliazation.py
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate

bs = 64  #batch size
sz = 224 #image size wanted
PATH = "/content/drive/My Drive/Auto_Query/downloads"

In [0]:
# google_images_download library
!pip install google_images_download
import re
from google_images_download import google_images_download


In [0]:
response = google_images_download.googleimagesdownload()
# TODO: Take in all queries at once, separated by commas, then split into
# individual queries
request_one = input('What would you like first?')
request_two = input('And then?')
name_one = str(request_one)
name_two = str(request_two)
#Test
print('first request: ' +request_one)
print('second request: '+request_two)

search_queries = [request_one, request_two]
print(search_queries)

def downloadimages(query):  
  arguments = {"keywords":query,
              "format": "jpg",
#               "limit": 250,
              "print_urls": True,
              "size": "medium"}
  try:
    response.download(arguments)
    
  except FileNotFoundError:  
        arguments = {"keywords": query, 
                     "format": "jpg", 
#                      "limit":4, 
                     "print_urls":True,  
                     "size": "medium", 
                     "usage_rights":"labeled-for-reuse"} 
                       
        try: 
            # Downloading the photos based 
            # on the given arguments 
            response.download(arguments)  
        except: 
            pass
  
for query in search_queries:
  downloadimages(query)
  print()



In [0]:
#get_classes.py

classes = []
for d in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, d)) and not d.startswith('.'):
        classes.append(d) 
print ("There are ", len(classes), "classes:\n", classes) 

In [0]:
#verify_images.py

for c in classes:
    print ("Class:", c)
    verify_images(os.path.join(PATH, c), delete=True);

In [0]:
#create_training_validation.py
data  = ImageDataBunch.from_folder(PATH, ds_tfms=get_transforms(), size=sz, bs=bs, valid_pct=0.2).normalize(imagenet_stats)
print ("There are", len(data.train_ds), "training images and", len(data.valid_ds), "validation images." )

In [0]:
#show data
data.show_batch(rows = 3, figsize=(7,8))

In [0]:
#If the user wants to build their own model
# import tensorflow as tf
# import keras
custom_query = input('Would you like to build your own model?')

if custom_query.lower() == 'yes':
  print('true')
#TODO: request model specifications and use keras to build it out
  

In [0]:
#TODO: Rename the individual files by the class later, if needed
# def renaming(name_one, name_two):
#   i = 0
#   j = 0
#   folder_one = "/content/drive/My Drive/Auto_Query/downloads/" + str(name_one) 
#   folder_two = "/content/drive/My Drive/Auto_Query/downloads/" + str(name_two)
  
#   for filename in os.listdir(folder_one):
#     prevname = str(filename)
#     newname = folder_one + str(name_one) + '/' + str(name_one) + str(i) + '.jpg'
#     print('checking...')
# #     dst = '{}'.format(name_one) + str(i) + '.jpg'
# #     src = folder_one + filename 
# #     dst = folder_one + dst 
# #     os.rename(src, dst)
#     os.rename(prevname, newname)
# #     src = folder_one + filename
# #     dst = folder_one + dst
# #     os.rename(src, dst)
#     i += 1
#   print('Finished First Query')

#   for filename in os.listdir(folder_two):
#     print('checking..')
#     newname = '{}'.format(name_one) + str(i) + '.jpg'
#     os.rename(filename, newname)
# #     dst = '{}'.format(name_one) + str(j) + '.jpg'
# #     src = folder_two + filename
# #     dst = folder_two + dst
# #     os.rename(src, dst)
#     j += 1
#   print('Finished Second Query')
# renaming(name_one, name_two)


In [0]:
#model
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.lr_find();
learn.recorder.plot()

In [0]:
#fit
#adjust to ideal learning rate manually for now
learn.fit_one_cycle(7, max_lr=slice(1e-3,1e-2))

In [0]:
#interpretation
interpretation = ClassificationInterpretation.from_learner(learn)
interpretation.plot_confusion_matrix(figsize=(12,12), dpi=60)
interpretation.plot_top_losses(9, figsize=(15,11), heatmap=False)

In [0]:
# test model with a new image
# TODO: either set up a new image scrape and make sure the image is new, 
# or connect to a webcam on local machine

# path = './'
# img = open_image(get_image_files(path)[0])
# pred_class,pred_idx,outputs = learn.predict(img)
# img.show()
# print ("It is a", pred_class)